In [1]:
import pandas as pd
import xlrd
import os
import random

# Data exploration


In [3]:
for path in [
    "data-1/raw/dds/Groupage/",
    "data-1/raw/dds/Innight/",
    "data-1/raw/dds/Pallet/",
    "data-1/raw/dds/Road_Freight/"
]:
    files = os.listdir(path)
    print(len(files))
    chosen_file = random.choice(files)
    workbook = xlrd.open_workbook(f"{path}/{chosen_file}")
    data = pd.read_excel(workbook)
    # print(data.info())


32
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
32
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
32
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
4
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [4]:
len(data.columns)

25

# Preprocess data, combine data and write to parquet

In [ ]:
# Assume data is store in data/raw/
def combine_data(data_path, data_name):
    data = pd.DataFrame()
    for file_name in os.listdir(data_path):
        file_path = f"{data_path}{file_name}"
        workbook = xlrd.open_workbook(file_path, logfile=open(os.devnull, "w"))
        data = pd.concat([data, pd.read_excel(workbook)])
        data.to_parquet(f"data/raw/{data_name}.parquet")
    
    return data

groupage_data = combine_data("data-1/raw/dds/Groupage/", "groupage_data")
innight_data = combine_data("data-1/raw/dds/Innight/", "innight_data")
pallet_data = combine_data("data-1/raw/dds/Pallet/", "pallet_data")
road_freight_data = combine_data("data-1/raw/dds/Road_Freight/", "road_freight_data")

customer_data = pd.read_excel("data-1/raw/master-data/ADS_DDS_RF_CustomerNumbers_MasterData.xlsx")
customer_data.to_parquet("data-1/raw/customer_data.parquet")

In [2]:
# Load data
groupage_data = pd.read_parquet("data-1/raw/groupage_data.parquet")
innight_data = pd.read_parquet("data-1/raw/innight_data.parquet")
pallet_data = pd.read_parquet("data-1/raw/pallet_data.parquet")
road_freight_data = pd.read_parquet("data-1/raw/road_freight_data.parquet")
customer_data = pd.read_parquet("data-1/raw/customer_data.parquet")

In [3]:
# combine data
combined_data = pd.concat([groupage_data, innight_data, pallet_data, road_freight_data])

# check na
combined_data.isna().sum()

Customer_ID                                   9654
ConsignorCountryCode                             0
Consignor Location Nr                            0
ConsigneeCountryCode                             0
Consignee Location Nr                            0
Freight calc_ status                             0
Product                                          0
Parcel/Pallet type                         2268022
Shipment Tracking Number                         0
ShipmentDate                                     0
ConsignorParcelCount                             0
ConsignorWeight                              57705
Freight                                          0
Surcharge                                        0
Dangerous goods surcharge                        0
Delivery outside of core area surcharge          0
Energy surcharge                                 0
Infrastructure surcharge                         0
Multi-parcel surcharge                           0
Private delivery surcharge     

In [10]:
combined_data.shape

(3234869, 25)

In [4]:
# fill Parcel/Pallet type with "PLL" if it is na
combined_data["Parcel/Pallet type"] = combined_data["Parcel/Pallet type"].fillna("PLL")
if "FileName" in combined_data.columns: 
    combined_data = combined_data.drop(columns=["FileName"])

# remove duplicate, keep first
print(f"Number of duplicate rows: {combined_data.duplicated().sum()}")
combined_data = combined_data.drop_duplicates(keep="first")

# now we can drop duplicates in Shipment Tracking Number
print(f"Number of duplicate rows in STN: {combined_data.duplicated(subset=['Shipment Tracking Number']).sum()}")
combined_data = combined_data.drop_duplicates(subset=["Shipment Tracking Number"])

Number of duplicate rows: 1504
Number of duplicate rows in STN: 205


In [10]:
# unique postcode
combined_data["Consignee Location Nr"].nunique()

11354

In [16]:
# load customer data
customer_data = pd.read_excel("data-1/raw/master-data/ADS_DDS_RF_CustomerNumbers_MasterData.xlsx")
print(customer_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4229 entries, 0 to 4228
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Customer ID           4229 non-null   object 
 1   Main Customer         4229 non-null   object 
 2   Industry Level 1      4229 non-null   object 
 3   Industry Level 2      4229 non-null   object 
 4   System                4229 non-null   object 
 5   Allocated SAP Number  4202 non-null   float64
dtypes: float64(1), object(5)
memory usage: 198.4+ KB
None


In [17]:
customer_data.columns

Index(['Customer ID', 'Main Customer', 'Industry Level 1', 'Industry Level 2',
       'System', 'Allocated SAP Number'],
      dtype='object')

In [9]:
print(combined_data.shape)
print(combined_data[combined_data["Customer_ID"].isin(customer_data["Customer ID"])].shape)
print(
    combined_data.shape[0] - combined_data[combined_data["Customer_ID"].isin(customer_data["Customer ID"])].shape[0] 
)

(3233160, 24)
(3222775, 24)
10385


In [10]:
combined_data["Customer_ID"].nunique()

4024

In [11]:
customer_data["Customer ID"].nunique()

4229

In [12]:
combined_data[combined_data["Customer_ID"].isin(customer_data["Customer ID"])].isna().sum()

Customer_ID                                    0
ConsignorCountryCode                           0
Consignor Location Nr                          0
ConsigneeCountryCode                           0
Consignee Location Nr                          0
Freight calc_ status                           0
Product                                        0
Parcel/Pallet type                             0
Shipment Tracking Number                       0
ShipmentDate                                   0
ConsignorParcelCount                           0
ConsignorWeight                            57521
Freight                                        0
Surcharge                                      0
Dangerous goods surcharge                      0
Delivery outside of core area surcharge        0
Energy surcharge                               0
Infrastructure surcharge                       0
Multi-parcel surcharge                         0
Private delivery surcharge                     0
Road toll           

In [13]:
# drop na in Customer_ID
combined_data = combined_data.dropna(subset=["Customer_ID"])

# merge combined data and customer data
combined_data = pd.merge(combined_data, customer_data, left_on="Customer_ID", right_on="Customer ID", how="left")

combined_data = combined_data.drop(columns=["Customer ID"])

In [ ]:
# Load DK postcode data
dk_postcode = pd.read_csv("data-1/raw/master-data/New_PostCodes_Data_DK/DK_Tableau.csv", delimiter=";", decimal=",")

In [ ]:
# merge combined data and dk postcode data
combined_data = pd.merge(combined_data, dk_postcode, left_on="Consignee Location Nr", right_on="Consignee Location Nr", how="left")

In [23]:
combined_data = combined_data.drop(columns=["Consignee Location Nr_y"], axis=1)
combined_data = combined_data.drop(columns=["ConsigneeCountryCode_y"], axis=1)
combined_data = combined_data.rename(columns={"Consignee Location Nr_x": "Consignee Location Nr", "ConsigneeCountryCode_x": "ConsigneeCountryCode"})

Index(['Customer_ID', 'ConsignorCountryCode', 'Consignor Location Nr',
       'ConsigneeCountryCode_x', 'Consignee Location Nr',
       'Freight calc_ status', 'Product', 'Parcel/Pallet type',
       'Shipment Tracking Number', 'ShipmentDate', 'ConsignorParcelCount',
       'ConsignorWeight', 'Freight', 'Surcharge', 'Dangerous goods surcharge',
       'Delivery outside of core area surcharge', 'Energy surcharge',
       'Infrastructure surcharge', 'Multi-parcel surcharge',
       'Private delivery surcharge', 'Road toll', 'Special goods surcharge',
       'Price_Paid', 'Base_Price', 'Main Customer', 'Industry Level 1',
       'Industry Level 2', 'System', 'Allocated SAP Number',
       'ConsigneeCountryCode_y', 'Latitude (generated)',
       'Longitude (generated)'],
      dtype='object')

In [24]:
combined_data[:3]

,Customer_ID,ConsignorCountryCode,Consignor Location Nr,ConsigneeCountryCode_x,Consignee Location Nr,Freight calc_ status,Product,Parcel/Pallet type,Shipment Tracking Number,ShipmentDate,...,Price_Paid,Base_Price,Main Customer,Industry Level 1,Industry Level 2,System,Allocated SAP Number,ConsigneeCountryCode_y,Latitude (generated),Longitude (generated)
0,CGDAACEIA,DK,2690,NO,1254,10,G,PLL,5DO5Z8COOAZ4CDAB8,2017-08-02,...,218.896,596.8,Customer 389,Wholesaler,Wholesale/Construction,DDS,240182871.0,DK,55.6812,12.5903
1,CGDAACEIA,DK,2690,NO,1187,10,G,PLL,5DO5Z8COOAZ6Z6DBC,2017-08-08,...,218.896,596.8,Customer 389,Wholesaler,Wholesale/Construction,DDS,240182871.0,NaN,NaN,NaN
2,CGDAACEIA,DK,2690,NO,1184,10,G,PLL,5DO5Z8COOAZ6Z6ZZO,2017-08-08,...,218.896,596.8,Customer 389,Wholesaler,Wholesale/Construction,DDS,240182871.0,NaN,NaN,NaN


In [ ]:
combined_data = pd.read_parquet("data-1/processed/combined_data.parquet")
consignee_generated_coords = pd.read_csv("data-1/raw/master-data/consignee_generated_coords.csv")
consignor_generated_coords = pd.read_csv("data-1/raw/master-data/consignor_generated_coords.csv")

In [ ]:
missing_consignee_coords = consignee_generated_coords[consignee_generated_coords["Latitude (generated)"].isna()]
missing_consignor_coords = consignor_generated_coords[consignor_generated_coords["Latitude (generated)"].isna()]

missing_coords = pd.concat([missing_consignee_coords, missing_consignor_coords])

missing_coords = missing_coords.drop_duplicates()


In [ ]:
missing_consignee_coords = missing_consignee_coords.to_dict(orient="records")


,Consignee Country Code,Consignee Location Nr,Latitude (generated),Longitude (generated)
27,SE,89189,NaN,NaN
45,SE,89104,NaN,NaN
66,SE,88104,NaN,NaN
89,SE,87222,NaN,NaN
90,SE,87200,NaN,NaN
...,...,...,...,...
11782,DK,899,NaN,NaN
11783,DK,894,NaN,NaN
11784,DK,892,NaN,NaN
11785,DK,877,NaN,NaN


In [ ]:
dk_postcode = pd.read_excel("data-1/raw/master-data/New_PostCodes_Data_DK/DK_Geonames.xlsx")

In [ ]:
norway_postcode = pd.read_excel("data-1/raw/master-data/Postnummerregister-Excel.xlsx")
norway_postcode = norway_postcode.to_dict(orient="records")

denmark_postcode = pd.read_excel("data-1/raw/master-data/postcodes-file-to-download.xls", header=1)
denmark_postcode = denmark_postcode.to_dict(orient="records")

In [2]:
combined_data = pd.read_parquet("data-1/processed/combined_data.parquet")
consignee_generated_coords = pd.read_csv("data-1/raw/master-data/consignee_generated_coords.csv")
consignor_generated_coords = pd.read_csv("data-1/raw/master-data/consignor_generated_coords.csv")

In [25]:
consignor_generated_coords = consignor_generated_coords.rename(
    columns={
        "Latitude (generated)":"ConsignorLatitude",
        "Longitude (generated)":"ConsignorLongitude"
    }
)
consignee_generated_coords = consignee_generated_coords.rename(
    columns={
        "Latitude (generated)":"ConsigneeLatitude",
        "Longitude (generated)":"ConsigneeLongitude"
    }
)
consignor_generated_coords["Consignor Location Nr"] = consignor_generated_coords["Consignor Location Nr"].astype(str).str.zfill(4)
consignee_generated_coords["Consignee Location Nr"] = consignee_generated_coords["Consignee Location Nr"].astype(str).str.zfill(4)

In [26]:
combined_data = pd.merge(combined_data, consignor_generated_coords, left_on=["ConsignorCountryCode", "Consignor Location Nr"], right_on=["Consignor Country Code", "Consignor Location Nr"], how="left")
combined_data = pd.merge(combined_data, consignee_generated_coords, left_on=["ConsigneeCountryCode", "Consignee Location Nr"], right_on=["Consignee Country Code", "Consignee Location Nr"], how="left")

In [27]:
combined_data.to_csv("data-1/processed/combined_data_with_coords.csv", index=False)